###  Introduction
This notebook covers the initial data cleaning and wrangling process necessary to produce a single dataset of each food's nutritional values, which can be searched via a food's description.

In [34]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [35]:
pd.set_option('max_columns', 300)
pd.set_option('max_rows', 5000)
pd.set_option('display.max_colwidth', None)

In [36]:
# Importing necessary CSVs, ignoring mixed dtype warning as specified column will not be used.
nutrient_names = pd.read_csv('./New_CSVs/nutrient.csv')
nutrient_values = pd.read_csv('./New_CSVs/food_nutrient.csv')
food_names = pd.read_csv('./New_CSVs/food.csv')

/home/james/miniconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [37]:
print(nutrient_names.shape)
nutrient_names.head(5)

(237, 5)


,id,name,unit_name,nutrient_nbr,rank
0,1002,Nitrogen,G,202.0,500.0
1,1003,Protein,G,203.0,600.0
2,1004,Total lipid (fat),G,204.0,800.0
3,1005,"Carbohydrate, by difference",G,205.0,1110.0
4,1007,Ash,G,207.0,1000.0


'nutrient_names' contains the name and units for each of the 237 nutrients available for each food

In [38]:
print(nutrient_values.shape)
nutrient_values.head(5)

(7547844, 11)


,id,fdc_id,nutrient_id,amount,data_points,derivation_id,min,max,median,footnote,min_year_acquired
0,13338591,323505,2048,35.0,NaN,49.0,NaN,NaN,NaN,NaN,NaN
1,13338389,326196,2048,36.0,NaN,49.0,NaN,NaN,NaN,NaN,NaN
2,13338390,747447,2048,32.0,NaN,49.0,NaN,NaN,NaN,NaN,NaN
3,13338391,321611,2048,20.0,NaN,49.0,NaN,NaN,NaN,NaN,NaN
4,13338392,746769,2048,17.0,NaN,49.0,NaN,NaN,NaN,NaN,NaN


'nutrient_values' contains the value of each nutrient for each food and a 'nutrient_id' key that links to the 'nutrient_names' dataset

In [39]:
print(food_names.shape)
food_names.head(5)

(533612, 5)


,fdc_id,data_type,description,food_category_id,publication_date
0,1105898,experimental_food,Metabolizable Energy of Almonds,NaN,2020-10-30
1,1105899,experimental_food,"Metabolizable Energy of Almonds, Food Processing and Structure Impact",NaN,2020-10-30
2,1105900,experimental_food,Metabolizable Energy of Cashews,NaN,2020-10-30
3,1105901,experimental_food,Metabolizable Energy of Pistachios,NaN,2020-10-30
4,1105902,experimental_food,Metabolizable Energy of Walnuts,NaN,2020-10-30


'food names' contains the descriptions for each food

In [40]:
print(food_names.data_type.unique())

['experimental_food' 'sr_legacy_food' 'sample_food' 'market_acquistion'
 'sub_sample_food' 'foundation_food' 'agricultural_acquisition'
 'branded_food' 'survey_fndds_food']


** For conciseness, a portion of code has been removed exploring the foods found in each food type and the availability of nutritional info for each, as well as exploring which of the 237 nutrients are most widely available across all the foods. **

<br>

---

<br>

### Data wrangling

The goal is to create a single dataset containing all the useful foods with correctly labelled nutritional information for each. <br>Several steps must be taken to achieve this: 
- Out of the 533,612 foods, remove any that are not relevant to our application such as experimental foods or repeated samplings of the same foods.
- To avoid an overwhelming amount of information, decide which nutrients are most important to include out the 237 available.
- Join each food with its nutritional information and correctly label the name and unit of each nutrient.

In [41]:
# Selecting food_types with useful nutritional information, selecting relevant columns.
food_names = food_names.dropna(subset=['description'])
food_names = food_names[food_names.data_type.isin(['branded_food', 'sr_legacy_food', 'survey_fndds_food'])]
food_names = food_names[['fdc_id', 'description']]
food_names.description = food_names.description.str.lower()

In [42]:
# Selecting relevant columns, pivoting so each nutrient_id is a column.
nutrient_values = nutrient_values.iloc[:, 1:4]
nutrient_values = nutrient_values.pivot(index='fdc_id', columns='nutrient_id', values='amount')

# Selecting relevant nutrients with using their nutrient_IDs, removing rows for foods without a description.
nutrient_values = nutrient_values[
    [1008,1051,1057,1018,1005,1079,1009,2000,1004,1292,1293,1258,1257,1253,1003,1221,
     1212,1213,1214,1215,1217,1211,1210,1219,1165,1166,1167,1170,1175,1177,1178,1180,
     1104,1162,1110,1109,1185,1087,1098,1100,1089,1090,1101,1091,1092,1103,1093,1095]].reset_index()
nutrient_values.columns.name = None
nutrient_values = nutrient_values[nutrient_values.fdc_id.isin(food_names.fdc_id) == True]

In [43]:
# Combining food descriptions with their nutrient values.
df = nutrient_values.set_index('fdc_id').join(food_names.set_index('fdc_id')).reset_index()

In [44]:
# Using nutrient IDs to select columns of nutrient_names and rename with nutrient names with units.
nutrient_names = nutrient_names.set_index('id').loc[df.columns[1:-1]]

nutrient_names.loc[[1292, 1293, 1258,1257], 'name'] = ['Monounsaturated fat', 'Polyunsaturated fat', 
                                                       'Saturated fat', 'Trans fat']

name_list = [name.split(',')[0].split('(')[0] + ' ({})'.format(unit.lower()) for 
             name, unit in zip(nutrient_names.name, nutrient_names.unit_name)]

name_list = ['fdc_id'] + name_list + ['Description']
df.columns = name_list

In [45]:
df['Completeness (%)'] = df.iloc[:, 1:-1].apply(lambda x: (x.notnull().sum() / 48) * 100, axis=1)
df['Completeness (%)'] = df['Completeness (%)'].round(1)

In [46]:
print(df.shape)
df.head(5)

(498824, 51)


,fdc_id,Energy (kcal),Water (g),Caffeine (mg),Alcohol (g),Carbohydrate (g),Fiber (g),Starch (g),Sugars (g),Total lipid (g),Monounsaturated fat (g),Polyunsaturated fat (g),Saturated fat (g),Trans fat (g),Cholesterol (mg),Protein (g),Histidine (g),Isoleucine (g),Leucine (g),Lysine (g),Methionine (g),Phenylalanine (g),Threonine (g),Tryptophan (g),Valine (g),Thiamin (mg),Riboflavin (mg),Niacin (mg),Pantothenic acid (mg),Vitamin B-6 (mg),Folate (ug),Vitamin B-12 (ug),Choline (mg),Vitamin A (iu),Vitamin C (mg),Vitamin D (iu),Vitamin E (mg),Vitamin K (ug),Calcium (mg),Copper (mg),Iodine (ug),Iron (mg),Magnesium (mg),Manganese (mg),Phosphorus (mg),Potassium (mg),Selenium (ug),Sodium (mg),Zinc (mg),Description,Completeness (%)
0,167512,307.0,35.50,NaN,NaN,41.18,1.2,NaN,5.88,13.24,NaN,NaN,2.941,4.412,0.0,5.88,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.12,NaN,NaN,NaN,NaN,NaN,1059.0,NaN,"pillsbury golden layer buttermilk biscuits, artificial flavor, refrigerated dough",25.0
1,167513,330.0,27.86,NaN,NaN,53.42,1.4,NaN,21.34,11.27,NaN,NaN,3.250,4.290,0.0,4.34,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.1,NaN,NaN,NaN,28.0,NaN,NaN,1.93,NaN,NaN,NaN,NaN,NaN,780.0,NaN,"pillsbury, cinnamon rolls with icing, refrigerated dough",31.2
2,167514,377.0,3.20,NaN,NaN,79.80,NaN,NaN,NaN,3.70,NaN,NaN,NaN,NaN,NaN,6.10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2182.0,NaN,"kraft foods, shake n bake original recipe, coating for pork, dry",12.5
3,167515,232.0,42.60,NaN,NaN,46.00,NaN,NaN,NaN,1.80,0.303,0.920,0.308,0.082,NaN,8.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,180.0,NaN,NaN,1.40,NaN,NaN,NaN,NaN,NaN,345.0,NaN,"george weston bakeries, thomas english muffins",29.2
4,167516,273.0,40.34,0.0,0.0,41.05,2.2,35.51,4.30,9.22,4.530,1.445,1.898,NaN,15.0,6.58,0.143,0.281,0.489,0.296,0.133,0.304,0.225,0.074,0.326,0.499,0.665,6.675,0.24,0.972,63.0,2.86,NaN,1340.0,0.0,NaN,0.62,10.5,279.0,0.04,NaN,6.04,19.0,0.215,388.0,126.0,9.5,621.0,0.48,"waffles, buttermilk, frozen, ready-to-heat",91.7


In [47]:
#df.to_csv('nutrition_data.csv')